<a href="https://colab.research.google.com/github/nazuna2371/ai_synthesized_voice/blob/main/tacotron2_train_side.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title **<font color= "blue">ドライブマウント・認証</font>**

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title **<font color= "blue">システム確認</font>**

!uname -a
print()
!cat /etc/os-release
print()
!nvidia-smi

Linux 5bbe58e688f7 5.15.109+ #1 SMP Fri Jun 9 10:57:30 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux

NAME="Ubuntu"
VERSION="20.04.6 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.6 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal

Mon Jul 17 11:41:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|========

In [3]:
#@title **<font color= "blue">python version 切り替え</font>**

!python --version
print()
!pip --version

Python 3.10.12

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!sudo apt install python3.7
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 3
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

!sudo apt-get -y install python3-pip
!sudo apt-get install --reinstall python3.7-distutils

In [5]:
!python --version
print()
!pip list

Python 3.7.17

Package             Version             
------------------- --------------------
dbus-python         1.2.16              
pip                 20.0.2              
PyGObject           3.36.0              
python-apt          2.0.1+ubuntu0.20.4.1
requests-unixsocket 0.2.0               
setuptools          45.2.0              
wheel               0.34.2              


In [6]:
#@title **<font color= "blue">install cuda10.0</font>**

!ls -d /usr/local/cuda-*
print()
!nvcc -V

/usr/local/cuda-11  /usr/local/cuda-11.8

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
# install cuda10.0
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo apt-get update
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt install -y ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!sudo apt-get update


# Install NVIDIA driver
#!sudo apt-get install --no-install-recommends nvidia-driver-418
!sudo apt-get -y installnvidia-driver-418
# Reboot. Check that GPUs are visible using the command: nvidia-smi

# Install development and runtime libraries (~4GB)
#!sudo apt-get install --no-install-recommends \
!sudo apt-get install -y \
    cuda-10-0 \
    libcudnn7=7.6.2.24-1+cuda10.0  \
    libcudnn7-dev=7.6.2.24-1+cuda10.0


# Install TensorRT. Requires that libcudnn7 is installed above.
# !sudo apt-get install -y --no-install-recommends libnvinfer5=5.1.5-1+cuda10.0 \
!sudo apt-get install -y libnvinfer5=5.1.5-1+cuda10.0 \
    libnvinfer-dev=5.1.5-1+cuda10.0

!apt --fix-broken install

In [8]:
!ls -d /usr/local/cuda-*
print()
!nvcc -V

/usr/local/cuda-10.0  /usr/local/cuda-11  /usr/local/cuda-11.8

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [ ]:
#@title **<font color= "blue">tensorflow等の準備</font>**

!pip install Unidecode==1.0.22
!pip install librosa==0.8.0
!pip install pysoundfile==0.9.0.post1
!pip install tensorflow-gpu==1.13.2
import sys
sys.path.insert(0,'/usr/local/lib/python3.7/dist-packages')
!pip install numpy==1.18.4
!pip install pandas==1.0.3
!pip install matplotlib==3.0.3
!pip install keras==2.2.4
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install inflect
!pip uninstall tensorflow
!pip install tensorflow==1.15

In [ ]:
#@title **<font color= "blue">git clone系</font>**

# NVIDIA/tacotron2のインストール
!git clone https://github.com/NVIDIA/tacotron2.git
%cd tacotron2
!git submodule init
!git submodule update

# Apexのインストール
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..

In [ ]:
!pip install -r requirements.txt

In [12]:
#@title **<font color= "blue">データ準備関連</font>**

!cp -p "/content/drive/MyDrive/work/test_data/corpus/yasunokiyono/filelists/transcript.txt" /content/tacotron2/filelists/transcript.txt
!wc -l /content/tacotron2/filelists/transcript.txt

38 /content/tacotron2/filelists/transcript.txt


In [13]:
!ls -ltr /content/tacotron2/filelists

total 1568
-rw------- 1 root root    3749 Feb 28 12:03 transcript.txt
-rwxr-xr-x 1 root root   60834 Jul 17 11:57 ljs_audio_text_test_filelist.txt
-rwxr-xr-x 1 root root   11911 Jul 17 11:57 ljs_audio_text_val_filelist.txt
-rwxr-xr-x 1 root root 1524164 Jul 17 11:57 ljs_audio_text_train_filelist.txt


In [14]:
!head -n 33 filelists/transcript.txt > filelists/transcript_train.txt
!tail -n 5 filelists/transcript.txt > filelists/transcript_val.txt
!wc -l /content/tacotron2/filelists/transcript_train.txt
!wc -l /content/tacotron2/filelists/transcript_val.txt

33 /content/tacotron2/filelists/transcript_train.txt
5 /content/tacotron2/filelists/transcript_val.txt


In [15]:
!ls -ltr /content/tacotron2/filelists

total 1576
-rw------- 1 root root    3749 Feb 28 12:03 transcript.txt
-rwxr-xr-x 1 root root   60834 Jul 17 11:57 ljs_audio_text_test_filelist.txt
-rwxr-xr-x 1 root root   11911 Jul 17 11:57 ljs_audio_text_val_filelist.txt
-rwxr-xr-x 1 root root 1524164 Jul 17 11:57 ljs_audio_text_train_filelist.txt
-rw-r--r-- 1 root root    3289 Jul 17 11:58 transcript_train.txt
-rw-r--r-- 1 root root     460 Jul 17 11:58 transcript_val.txt


In [16]:
%cd "/content/tacotron2"

# checkpoint 保存場所 シンボリックリンク
!ln -s "/content/drive/MyDrive/Colab Notebooks/model_storage/self_create/tacotron2/outdir" .
!ln -s "/content/drive/MyDrive/Colab Notebooks/model_storage/self_create/tacotron2/outdir_2" .

!ln -s "/content/drive/MyDrive/work/test_data/corpus/yasunokiyono/filelists/meian" .
!ls "/content/tacotron2/meian" -1U | wc

#!unlink outdir

/content/tacotron2
     38      38     950


In [17]:
!ls -lah .

total 1.1M
drwxr-xr-x 7 root root 4.0K Jul 17 11:59 .
drwxr-xr-x 1 root root 4.0K Jul 17 11:57 ..
drwxr-xr-x 9 root root 4.0K Jul 17 11:57 apex
-rw-r--r-- 1 root root 2.6K Jul 17 11:57 audio_processing.py
-rw-r--r-- 1 root root 4.4K Jul 17 11:57 data_utils.py
-rwxr-xr-x 1 root root 386K Jul 17 11:57 demo.wav
-rw-r--r-- 1 root root 7.1K Jul 17 11:57 distributed.py
-rw-r--r-- 1 root root  385 Jul 17 11:57 Dockerfile
drwxr-xr-x 2 root root 4.0K Jul 17 11:58 filelists
drwxr-xr-x 9 root root 4.0K Jul 17 11:57 .git
-rw-r--r-- 1 root root   99 Jul 17 11:57 .gitmodules
-rw-r--r-- 1 root root 2.9K Jul 17 11:57 hparams.py
-rw-r--r-- 1 root root 331K Jul 17 11:57 inference.ipynb
-rw-r--r-- 1 root root 3.0K Jul 17 11:57 layers.py
-rw-r--r-- 1 root root 1.5K Jul 17 11:57 LICENSE
-rw-r--r-- 1 root root 2.0K Jul 17 11:57 logger.py
-rw-r--r-- 1 root root  673 Jul 17 11:57 loss_function.py
-rw-r--r-- 1 root root 4.3K Jul 17 11:57 loss_scaler.py
lrwxrwxrwx 1 root root   73 Jul 17 11:59 meian -> /content

In [19]:
#@title **<font color= "blue">学習実行</font>**

!rm "/content/tacotron2/hparams.py"
!cp -p "/content/drive/MyDrive/Colab Notebooks/model_storage/tmp_hparams_storage/hparams_5.py" "/content/tacotron2/hparams.py"

!python train.py --output_directory=outdir --log_directory=logdir -c "/content/tacotron2/outdir/20230301_yasuno_cp_6000" --warm_start

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

FP16 Run: False
Dynamic Loss Scaling: True
Distributed Run: False
cuDNN Enabled: True
cuDNN Benchmark: False
Warm starting model from checkpoint '/content/tacotron2/outdir/20230301_yasuno_cp_6000'
Epoch: 0
Train loss 0 0.104254 Grad Norm 0.279741 3.50s/it
Validation loss 0:  1.616289  
Saving model and optimizer state at iteration 0 to outdir/checkpoint_0
Train loss 1 0.122166 Grad Norm 2.378822 2.85s/it
Train loss 2 0.078373 Grad Norm 0.302038 1.97s/it
Train loss 3 0.113873 Grad Norm 3.373257 3.02s/it
Train loss 4 0.102337 Grad Norm 1.980123 3.63s/it
Train loss 5 0.094566 Grad Norm 0.694002 2.65s/it
Train loss 6 0.110095 Grad N

In [ ]:
!cp -p "/content/tacotron2/outdir/checkpoint_100" "/content/drive/MyDrive/Colab Notebooks/model_storage/self_create/20230301_yasuno_cp_6100"